# 1. Import and Install Dependencies

In [5]:
!pip install tensorflow==2.5 tensorflow-gpu==2.5 opencv-python mediapipe sklearn matplotlib


In [3]:
import cv2
import numpy as np
import os 
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [4]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image,model):
    image =cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [6]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)    

In [7]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,70,10),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             )    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             )    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             )    

In [8]:
cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        #print(results)
        draw_styled_landmarks(image,results)
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# 3.Extract keypoints

In [11]:
pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
face=np.array([[res.x,res.y,res.z] fddor res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [12]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose,face,lh,rh])

# 4. Setup folders for data collection

In [15]:
DATA_PATH=os.path.join('MP_Data')
actions=np.array(['hello','thanks','iloveyou'])
no_sequences=30
sequence_length=30


In [16]:
for action in actions :
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# 5. Collect keypoints

In [17]:
cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret,frame=cap.read()
                image,results=mediapipe_detection(frame,holistic)
                #print(results)
                draw_styled_landmarks(image,results)
                
                if frame_num ==0:
                    cv2.putText(image,"Starting Collection ",(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,f"Collecting frames for {action} video number {sequence} ",(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else:
                    cv2.putText(image,f"Collecting frames for {action} video number {sequence} ",(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                #cv2.imshow('OpenCV Feed',image)

                if cv2.waitKey(10) & 0xff==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create labels and features

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
label_map={label:num for num,label in enumerate(actions)}

In [20]:
sequences ,labels =[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [21]:
len(labels)

90

In [22]:
X=np.array(sequences)

In [23]:
X.shape

(90, 30, 1662)

In [24]:
y= to_categorical(labels).astype(int)

In [25]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [26]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

In [27]:
y_test.shape

(5, 3)

# 7. Build and train LSTM Neural Network

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.callbacks import TensorBoard

In [82]:
log_dir=os.path.join("Logs")
tb_callback=TensorBoard(log_dir=log_dir)

In [84]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [85]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["categorical_accuracy"])

In [86]:
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])

Epoch 1/100
3/3 [==============================] - 9s 1s/step - loss: 1.4494 - categorical_accuracy: 0.3176
Epoch 2/100
3/3 [==============================] - 1s 308ms/step - loss: 1.5017 - categorical_accuracy: 0.2588
Epoch 3/100
3/3 [==============================] - 1s 284ms/step - loss: 1.8391 - categorical_accuracy: 0.2588
Epoch 4/100
3/3 [==============================] - 1s 349ms/step - loss: 3.0849 - categorical_accuracy: 0.2824
Epoch 5/100
3/3 [==============================] - 1s 310ms/step - loss: 10.7424 - categorical_accuracy: 0.3294
Epoch 6/100
3/3 [==============================] - 1s 287ms/step - loss: 11.5532 - categorical_accuracy: 0.3529
Epoch 7/100
3/3 [==============================] - 1s 135ms/step - loss: 12.9625 - categorical_accuracy: 0.4000
Epoch 8/100
3/3 [==============================] - 0s 146ms/step - loss: 18.3348 - categorical_accuracy: 0.4235
Epoch 9/100
3/3 [==============================] - 0s 139ms/step - loss: 96.3912 - categorical_accuracy: 0.2471

3/3 [==============================] - 0s 130ms/step - loss: 21.0993 - categorical_accuracy: 0.4000
Epoch 74/100
3/3 [==============================] - 0s 123ms/step - loss: 13.6071 - categorical_accuracy: 0.3529
Epoch 75/100
3/3 [==============================] - 0s 135ms/step - loss: 27.8411 - categorical_accuracy: 0.3882
Epoch 76/100
3/3 [==============================] - 0s 123ms/step - loss: 33.4303 - categorical_accuracy: 0.3294
Epoch 77/100
3/3 [==============================] - 0s 157ms/step - loss: 18.3211 - categorical_accuracy: 0.3529
Epoch 78/100
3/3 [==============================] - 0s 128ms/step - loss: 23.2389 - categorical_accuracy: 0.3176
Epoch 79/100
3/3 [==============================] - 0s 138ms/step - loss: 26.6675 - categorical_accuracy: 0.3765
Epoch 80/100
3/3 [==============================] - 0s 122ms/step - loss: 40.2419 - categorical_accuracy: 0.2941
Epoch 81/100
3/3 [==============================] - 0s 133ms/step - loss: 30.0521 - categorical_accuracy: 0.2

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_11 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

# 8. model prediction

In [105]:
res=model.predict(X_test)


In [106]:
actions[np.argmax(res[1])]

'iloveyou'

In [107]:
actions[np.argmax(y_test[1])]

'thanks'

# 9. Save Weights

In [108]:
model.save('action.h5')

# 10.Evaluation using Confusion Matrix and Accuracy

In [109]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [110]:
yhat=model.predict(X_test)

In [111]:
ytrue=np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

In [112]:
ytrue

[0, 1, 0, 1, 1]

In [113]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[3., 0.],
        [2., 0.]],

       [[2., 0.],
        [3., 0.]],

       [[0., 5.],
        [0., 0.]]])

In [114]:
accuracy_score(ytrue,yhat)

0.0

# 11. Test in Real time

In [115]:
colors=[(245,117,16),(117,245,16),(16,117,245)]
def prob_viz(res,actions,input_frame,colors):
    output_frame=input_frame.copy()
    for num,prob in enumerate(res):
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [116]:
sequence=[]
sentence=[]
predictions=[]
threshold=0.5

cap=cv2.VideoCapture(0)
 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        #print(results)
        draw_styled_landmarks(image,results)
        
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[-30:]
        
        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)]!=sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else: 
                        sentence.append(actions[np.argmax(res)])

            if len(sentence)>5:
                sentence=sentence[-5:]
            image= prob_viz(res,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
iloveyou
iloveyou
iloveyou
thanks
thanks
thanks
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
thanks
thanks
thanks
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
thanks
thanks
thanks
thanks
iloveyou
thanks
iloveyou
iloveyou
iloveyou
thanks
thanks
thanks
thanks
thanks
thanks
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
iloveyou
ilo